# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface = await new_transport_interface("0.0.0.0", 0)
client_tiface.run()

server_tiface = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface.run()

17:29:31.868427 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
17:29:31.870201 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
17:29:31.870247 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
17:29:31.872013 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
17:29:31.893832 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fc98b25fa10>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

# Filter layer setup

In [5]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7fc98b01b6d0>,
 's': <FilterInterface at 0x7fc98b01bdf0>}

Find methods to run, for informational purposes.

In [6]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command', 'command_response', 'get_notifications', 'response', 'run']

### Run Client and Server Filter!

In [7]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

## Filter layer operation for Pings

### Change the server status to something meaningful

In [8]:
server_name = "Local Conwayste Server (notebook)"
server_version = "1.2.34.56"
player_count = 8675309
room_count = 9001
await server_fiface.command_response(FilterCmdW("changeserverstatus",
                                                server_name=server_name,
                                                server_version=server_version,
                                                player_count=player_count,
                                                room_count=room_count))

17:29:40.164022 [TRACE] - [F<-A,C] New command: ChangeServerStatus { server_version: Some("1.2.34.56"), player_count: Some(8675309), room_count: Some(9001), server_name: Some("Local Conwayste Server (notebook)") }


Accepted

### As a client, add server (both our local and Chococat) as ping endpoints

#### First, resolve DNS for Chococat.

In [9]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

'157.230.134.224'

In [10]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept, choco_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }

In [11]:
await client_fiface.command_response(filter_cmd)

17:29:45.939215 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }


Accepted

17:29:45.986273 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:45.986368 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 4763853426349078934 } }] }
17:29:45.986464 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 3104080523621741063 } }] }
17:29:45.986527 [TRACE] - [F<-T,R] Command Accepted
17:29:45.986536 [TRACE] - [F<-T,R] Command Accepted
17:29:45.986571 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 4763853426349078934 } }
17:29:45.986588 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:54020), took packet GetStatus { ping: PingPong { nonce

17:29:46.786586 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:46.786623 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 6 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 170569259621526503 } }] }
17:29:46.786670 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 7 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 15299581605095599930 } }] }
17:29:46.786682 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 170569259621526503 } }
17:29:46.786700 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:54020), took packet GetStatus { ping: PingPong { nonce: 170569259621526503 } }
17:29:46.786709 [INFO ] - [F] Sending Status packet PingPong { nonce: 1705692596

17:29:47.607584 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 14281639680403541219 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:47.607623 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 14281639680403541219 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:47.607635 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(10)
17:29:47.786552 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:47.786593 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 10 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 9946671511414812402 } }] }
17:29:47.786649 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: E

17:29:48.410395 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 7532251341158956411 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:48.410434 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 7532251341158956411 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:48.410448 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(10)
17:29:48.586191 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:48.586222 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 6 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 1884814712062099205 } }] }
17:29:48.586277 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endp

17:29:49.385967 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:49.385995 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 22 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 611792057085973577 } }] }
17:29:49.386038 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 23 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 6447547130078652077 } }] }
17:29:49.386056 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 611792057085973577 } }
17:29:49.386074 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:54020), took packet GetStatus { ping: PingPong { nonce: 611792057085973577 } }
17:29:49.386083 [TRACE] - [F<-T,R] Command Accepted
17:29:49.386084 [INFO ] - [

17:29:50.007784 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 8584413836880766758 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:50.007867 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 8584413836880766758 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:50.007895 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(10)
17:29:50.185769 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:50.185804 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 18 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 1178425509617919611 } }] }
17:29:50.185859 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: End

17:29:50.810506 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 16145153538153936707 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:50.810542 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 16145153538153936707 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:50.810555 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(10)
17:29:50.986321 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:50.986424 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 18 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 18014794835793579185 } }] }
17:29:50.986578 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: 

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

### Sleep for a bit

We need time to get some pings.

In [12]:
time.sleep(3.0)

17:29:51.185773 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:51.185825 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 33 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 1433689029263163083 } }] }
17:29:51.185928 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 34 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 6327162838814561251 } }] }
17:29:51.185945 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 1433689029263163083 } }
17:29:51.185987 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:54020), took packet GetStatus { ping: PingPong { nonce: 1433689029263163083 } }
17:29:51.186003 [INFO ] - [F] Sending Status packet PingPong { nonce: 143368

17:29:51.986293 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:51.986334 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 38 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 2682111299120475256 } }] }
17:29:51.986388 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 39 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 13293401358523305697 } }] }
17:29:51.986435 [TRACE] - [F<-T,R] Command Accepted
17:29:51.986442 [TRACE] - [F<-T,R] Command Accepted
17:29:51.986464 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 2682111299120475256 } }
17:29:51.986478 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:54020), took packet GetStatus { ping: PingPong { no

17:29:52.806839 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 5659681058337194041 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:52.806872 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 5659681058337194041 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:52.806880 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(10)
17:29:52.986553 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:52.986594 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 33 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 11724540801902826207 } }] }
17:29:52.986640 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: En

17:29:53.607644 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 10113880061806790465 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:53.607677 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 10113880061806790465 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:29:53.607686 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(10)
17:29:53.786308 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:53.786394 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 40 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 202675261113819557 } }] }
17:29:53.786540 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: En

### Clear ping endpoints

In [13]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

17:29:54.104102 [TRACE] - [F<-A,C] New command: ClearPingEndpoints
17:29:54.104123 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
17:29:54.104179 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:62013) }
17:29:54.104191 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(157.230.134.224:2016) }
17:29:54.104200 [TRACE] - [F<-T,R] Command Accepted
17:29:54.104205 [TRACE] - [F<-T,R] Command Accepted


Accepted

### Get notifications from client and server

Normally the App layer would be getting these continuously.

In [14]:
server_notifications = server_fiface.get_notifications()
server_notifications

[]

In [15]:
client_notifications = client_fiface.get_notifications()
len(client_notifications)

82

In [16]:
list(map(lambda n: n.variant, client_notifications))

['PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResult',
 'PingResu

In [17]:
def server_info(notif):
    assert notif.variant.lower() == 'pingresult'
    return {'latency_ms':notif.latency, 'server_name': notif.server_name, 'room_count': notif.room_count, 'REPR': repr(notif)}

In [18]:
server_info(client_notifications[-1])

{'latency_ms': 10,
 'server_name': 'Official Conwayste',
 'room_count': 1,
 'REPR': 'PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: Some(10), server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 }'}

In [19]:
server_info(client_notifications[-2])

17:29:58.986920 [INFO ] - [F<-T,N] Endpoint(127.0.0.1:54020) timed-out. Dropping.
17:29:58.987243 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:54020) }
17:29:58.987362 [TRACE] - [F<-T,R] Command Accepted


{'latency_ms': 0,
 'server_name': 'Local Conwayste Server (notebook)',
 'room_count': 9001,
 'REPR': 'PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: Some(0), server_name: "Local Conwayste Server (notebook)", server_version: "1.2.34.56", room_count: 9001, player_count: 8675309 }'}

Sleeping again....

In [20]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

### Now shutdown

In [21]:
time.sleep(1)
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))

17:30:28.048901 [TRACE] - [F<-A,C] New command: Shutdown { graceful: true }
17:30:28.048969 [INFO ] - [F] shutting down
